In [44]:
import pandas as pd

In [85]:
Items_and_Opportunities_df['Currency'].unique()

array(['EUR', 'USD'], dtype=object)

In [45]:
Business_Account_df = pd.read_excel('shared_data_read_only/Business Account.xlsx')
consumer_price_index_df = pd.read_excel('shared_data_read_only/consumer_price_index.xlsx')
Contacts_df = pd.read_excel('shared_data_read_only/Contacts.xlsx')
Invoice_and_Opportunities_df = pd.read_excel('shared_data_read_only/Invoice and Opportunities.xlsx')
Items_and_Opportunities_df = pd.read_excel('shared_data_read_only/Items and Opportunities.xlsx')
Non_Stock_Items_df = pd.read_excel('shared_data_read_only/Non-Stock Items.xlsx')
Opportunities_df = pd.read_excel('shared_data_read_only/Opportunities.xlsx')

In [6]:
all_df = [Business_Account_df,
          consumer_price_index_df,
          Contacts_df,
          Invoice_and_Opportunities_df,
          Items_and_Opportunities_df,
          Non_Stock_Items_df,
          Opportunities_df]

In [23]:
for df in all_df:
    print(df.shape)
    print(df.columns)
    print(df.info())
    print('\n')

(3483, 17)
Index(['Business Account', 'Customer Status', 'Type', 'Partner type', 'Class',
       'Class Name', 'Activity sector', 'Parent Business Account',
       'Owner Name', 'Follow-up with', 'Postal Code', 'City', 'State',
       'Country ID', 'Country Name', 'Created On', 'Last Modified On'],
      dtype='object')


(56, 2)
Index(['Date', 'consumer_price_index'], dtype='object')


(5511, 17)
Index(['ContactID', 'Active', 'Business Account', 'Class', 'Class Name',
       'Financing preference', 'Owner', 'Follow-up with', 'Postal Code',
       'City', 'State', 'Code pays', 'Country Name', 'Gender', 'Client Source',
       'Last Modified On', 'Created On'],
      dtype='object')


(3238, 6)
Index(['Type', 'Reference Nbr.', 'Customer', 'Amount', 'Date',
       'Opportunity ID'],
      dtype='object')


(7458, 14)
Index(['Opportunity ID', 'Line Nbr.', 'Inventory ID', 'Year', 'Unit Price',
       'Quantity', 'UOM', 'Ext. Price', 'Currency', 'Discount Amount',
       'Discount, %', 'Fun

In [79]:
merge_1 = pd.merge(Opportunities_df, Business_Account_df, how='left', on='Business Account')
merge_2 = pd.merge(merge_1, Contacts_df, how='left', on='Business Account')
merge_3 = pd.merge(merge_2, Items_and_Opportunities_df, how='left', on='Opportunity ID')
merge_4 = pd.merge(merge_3, Invoice_and_Opportunities_df, how='left', on='Opportunity ID')
merge_df = pd.merge(merge_4, Non_Stock_Items_df, how='left', on='Inventory ID')

In [80]:
merge_df.shape

(38214, 101)

In [81]:
# Split the column based on commas and get a list of items
split_data = merge_df['Financing preference'].str.split(',', expand=True).apply(lambda x: x.str.strip())

# Gather all unique elements
unique_elements = pd.unique(split_data.values.ravel('K')).tolist()
unique_elements = [element for element in unique_elements if pd.notna(element)]

# Create a dictionary to hold the data for the new DataFrame
data_dict = {element: [] for element in unique_elements}

# Iterate through each row and populate the dictionary with 1 if the element is present, 0 otherwise
for index, row in split_data.iterrows():
    row_elements = row.dropna().tolist()
    for element in unique_elements:
        data_dict[element].append(1 if element in row_elements else 0)

# Create a new DataFrame using the dictionary
new_df = pd.DataFrame(data_dict)

In [83]:
concatenated_df.shape

(38214, 130)

In [82]:
concatenated_df = pd.concat([merge_df, new_df], axis=1)

In [ ]:
concatenated_df.to_csv('merged_transformed_split_financial_preference_into_cols.csv', index=False)